<a href="https://colab.research.google.com/github/Kavyapm1960/project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [22]:
import collections

import helper
import numpy as np
import project_tests as tests
from keras. preprocessing.text import Tokenizer
from keras. preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras. layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras. layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6115373970912935568
xla_global_id: -1
]


In [4]:
# Load English data
with open('/content/drive/MyDrive/small_vocab_en (2).txt', 'r', encoding='utf-8') as file:
    english_sentences = file.read().split('\n')

# Load French data
with open('/content/drive/MyDrive/small_vocab_fr (1).txt', 'r', encoding='utf-8') as file:
    french_sentences = file.read().split('\n')


In [5]:
# Load English data
english_sentences = helper.load_data('/content/drive/MyDrive/small_vocab_en (2).txt')
# Load French data
french_sentences = helper.load_data('/content/drive/MyDrive/small_vocab_fr (1).txt')

print('Dataset Loaded')

Dataset Loaded


In [ ]:
for sample_i in range(7):
    print('English sample {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('French sample {}:  {}\n'.format(sample_i + 1, french_sentences[sample_i]))

English sample 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
French sample 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .

English sample 2:  the united states is usually chilly during july , and it is usually freezing in november .
French sample 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .

English sample 3:  california is usually quiet during march , and it is usually hot in june .
French sample 3:  california est généralement calme en mars , et il est généralement chaud en juin .

English sample 4:  the united states is sometimes mild during june , and it is cold in september .
French sample 4:  les états-unis est parfois légère en juin , et il fait froid en septembre .

English sample 5:  your least liked fruit is the grape , but my least liked is the apple .
French sample 5:  votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .

English sample 6

In [ ]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
import tests  # Assuming you have a tests module

def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # Initialize tokenizer
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    # Convert text to sequences
    return tokenizer.texts_to_sequences(x), tokenizer

# Assuming tests.test_tokenize(tokenize) is defined in tests module

# Load first dataset from text file
file_path1 = '/content/drive/MyDrive/small_vocab_en (2).txt'
with open(file_path1, 'r') as file:
    text_data1 = file.readlines()

# Tokenize the first dataset
text_tokenized1, text_tokenizer1 = tokenize(text_data1)

# Load second dataset from text file
file_path2 = '/content/drive/MyDrive/small_vocab_fr (1).txt'
with open(file_path2, 'r') as file:
    text_data2 = file.readlines()

# Tokenize the second dataset
text_tokenized2, text_tokenizer2 = tokenize(text_data2)

# Print word index for the first dataset
print("Word Index for Dataset 1:")
print(text_tokenizer1.word_index)
print()

# Print tokenized sequences for the first dataset
print("Tokenized Sequences for Dataset 1:")
for sample_i, (sent, token_sent) in enumerate(zip(text_data1, text_tokenized1)):
    print('Sequence {} in Dataset 1'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))
    print()

# Print word index for the second dataset
print("Word Index for Dataset 2:")
print(text_tokenizer2.word_index)
print()

# Print tokenized sequences for the second dataset
print("Tokenized Sequences for Dataset 2:")

for sample_i, (sent, token_sent) in enumerate(zip(text_data2, text_tokenized2)):
    print('Sequence {} in Dataset 2'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))
    print()


Streaming output truncated to the last 5000 lines.

Sequence 16577 in Dataset 1
  Input:  china is usually pleasant during december , but it is chilly in fall .

  Output: [25, 1, 9, 52, 4, 47, 6, 3, 1, 62, 2, 33]

Sequence 16578 in Dataset 1
  Input:  india is sometimes dry during february , but it is never quiet in september .

  Output: [19, 1, 8, 59, 4, 48, 6, 3, 1, 10, 67, 2, 42]

Sequence 16579 in Dataset 1
  Input:  the lime is my favorite fruit , but the lemon is your favorite .

  Output: [5, 86, 1, 30, 11, 13, 6, 5, 85, 1, 29, 11]

Sequence 16580 in Dataset 1
  Input:  he dislikes grapes , limes , and strawberries.

  Output: [26, 69, 73, 71, 7, 79]

Sequence 16581 in Dataset 1
  Input:  france is never wet during july , but it is sometimes nice in november .

  Output: [24, 1, 10, 65, 4, 43, 6, 3, 1, 8, 41, 2, 45]

Sequence 16582 in Dataset 1
  Input:  paris is usually wonderful during december , but it is sometimes beautiful in summer .

  Output: [18, 1, 9, 58, 4, 47, 6, 3

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
Sequence 67494 in Dataset 1
  Input:  china is sometimes freezing during winter , and it is sometimes busy in autumn .

  Output: [25, 1, 8, 51, 4, 37, 7, 3, 1, 8, 60, 2, 39]

Sequence 67495 in Dataset 1
  Input:  the mango is their most loved fruit , but the apple is your most loved .

  Output: [5, 88, 1, 50, 14, 15, 13, 6, 5, 83, 1, 29, 14, 15]

Sequence 67496 in Dataset 1
  Input:  this bear was your favorite animal .

  Output: [124, 162, 104, 29, 11, 99]

Sequence 67497 in Dataset 1
  Input:  india is sometimes pleasant during august , but it is usually warm in spring .

  Output: [19, 1, 8, 52, 4, 61, 6, 3, 1, 9, 56, 2, 35]

Sequence 67498 in Dataset 1
  Input:  india is sometimes quiet during april , but it is usually nice in winter .

  Output: [19, 1, 8, 67, 4, 44, 6, 3, 1, 9, 41, 2, 37]

Sequence 67499 in Dataset 1
  Input:  her least favorite fruit is the lime , but his least favorite is the banana .

  Output: [32, 12, 11,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
  Output: [7, 80, 1, 40, 16, 10, 23, 14, 5, 7, 88, 1, 39, 23, 14]

Sequence 23113 in Dataset 2
  Input:  la france est parfois froid en février , mais il est parfois chaud en septembre .

  Output: [7, 33, 1, 8, 19, 2, 53, 5, 3, 1, 8, 21, 2, 48]

Sequence 23114 in Dataset 2
  Input:  paris est le gel en juillet , mais il est jamais beau en été .

  Output: [29, 1, 10, 81, 2, 49, 5, 3, 1, 9, 68, 2, 46]

Sequence 23115 in Dataset 2
  Input:  la france est généralement calme en juin , mais il est froid en octobre .

  Output: [7, 33, 1, 12, 67, 2, 41, 5, 3, 1, 19, 2, 54]

Sequence 23116 in Dataset 2
  Input:  le raisin est son fruit préféré moins , mais la mangue est notre moins préféré .

  Output: [10, 82, 1, 20, 16, 17, 13, 5, 7, 80, 1, 61, 13, 17]

Sequence 23117 in Dataset 2
  Input:  elle aime les pommes , les raisins et les fraises .

  Output: [27, 55, 4, 70, 4, 77, 6, 4, 74]

Sequence 23118 in Dataset 2
  Input:  new jersey est j

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
Sequence 136861 in Dataset 2
  Input:  les états-unis est jamais chaud au mois de mars , mais il est généralement agréable en juillet .

  Output: [4, 32, 31, 1, 9, 21, 15, 25, 22, 45, 5, 3, 1, 12, 18, 2, 49]

Sequence 136862 in Dataset 2
  Input:  l' inde est parfois agréable en juillet , mais il est parfois le gel en hiver .

  Output: [11, 30, 1, 8, 18, 2, 49, 5, 3, 1, 8, 10, 81, 2, 44]

Sequence 136863 in Dataset 2
  Input:  l'orange est son fruit préféré moins , mais la pomme est votre préféré moins .

  Output: [84, 1, 20, 16, 17, 13, 5, 7, 83, 1, 40, 17, 13]

Sequence 136864 in Dataset 2
  Input:  notre fruit est moins aimé la mangue , mais elle est moins aimé l'orange .

  Output: [61, 16, 1, 13, 14, 7, 80, 5, 27, 1, 13, 14, 84]

Sequence 136865 in Dataset 2
  Input:  la france est le gel habituellement au mois d' août , et il est parfois sec en juillet .

  Output: [7, 33, 1, 10, 81, 69, 15, 25, 79, 56, 6, 3, 1, 8, 63, 2, 49]


In [17]:
import numpy as np

# Assuming you already have tokenized data stored in 'text_tokenized'

# Pad Tokenized output
test_pad = pad(text_tokenized1)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized1, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))


Streaming output truncated to the last 5000 lines.
  Input:  [ 5 89  1 32 12 16 13  6  5 85  1 30 12 16]
  Output: [ 5 89  1 32 12 16 13  6  5 85  1 30 12 16  0]
Sequence 136195 in x
  Input:  [17 23  1  8 55  4 33  6  3  1  8 67  2 54]
  Output: [17 23  1  8 55  4 33  6  3  1  8 67  2 54  0]
Sequence 136196 in x
  Input:  [98 92 77 71  7 73]
  Output: [98 92 77 71  7 73  0  0  0  0  0  0  0  0  0]
Sequence 136197 in x
  Input:  [18  1  9 55  4 48  7  3  1 41  2 47]
  Output: [18  1  9 55  4 48  7  3  1 41  2 47  0  0  0]
Sequence 136198 in x
  Input:  [49 12 11 13  1  5 85  6 30 12 11  1  5 86]
  Output: [49 12 11 13  1  5 85  6 30 12 11  1  5 86  0]
Sequence 136199 in x
  Input:  [98 93 76 75  7 78]
  Output: [98 93 76 75  7 78  0  0  0  0  0  0  0  0  0]
Sequence 136200 in x
  Input:  [26 70 78 74  7 73]
  Output: [26 70 78 74  7 73  0  0  0  0  0  0  0  0  0]
Sequence 136201 in x
  Input:  [ 26 104 107 100  17 118 102]
  Output: [ 26 104 107 100  17 118 102   0   0   0   0   0   0 

In [18]:
import numpy as np

# Assuming you already have tokenized data stored in 'text_tokenized'

# Pad Tokenized output
test_pad = pad(text_tokenized2)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized2, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))


Streaming output truncated to the last 5000 lines.
   0   0   0]
Sequence 136374 in x
  Input:  [10 85  1 20 16 17 13  5  7 87  1 39 17 13]
  Output: [10 85  1 20 16 17 13  5  7 87  1 39 17 13  0  0  0  0  0  0  0]
Sequence 136375 in x
  Input:  [ 27 178  22 136  10 163 228   2  47]
  Output: [ 27 178  22 136  10 163 228   2  47   0   0   0   0   0   0   0   0   0
   0   0   0]
Sequence 136376 in x
  Input:  [107 245 159  78   6  28  76]
  Output: [107 245 159  78   6  28  76   0   0   0   0   0   0   0   0   0   0   0
   0   0   0]
Sequence 136377 in x
  Input:  [ 20 109  10  23  14 122 115 218]
  Output: [ 20 109  10  23  14 122 115 218   0   0   0   0   0   0   0   0   0   0
   0   0   0]
Sequence 136378 in x
  Input:  [35 34  1 12 19 26 98  6  3  1  9 57  2 41]
  Output: [35 34  1 12 19 26 98  6  3  1  9 57  2 41  0  0  0  0  0  0  0]
Sequence 136379 in x
  Input:  [101   1  12  18  15  42   6   3  95  69   2  56]
  Output: [101   1  12  18  15  42   6   3  95  69   2  56   0   0  

In [19]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)


max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [20]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [21]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 0.005

    model = Sequential()
    model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_simple_model(simple_model)

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print(simple_rnn_model.summary())


AttributeError: module 'tests' has no attribute 'test_simple_model'